In [1]:
import aw_client
import catboost


catboost.__version__

'1.2.7'

In [2]:
from aw_client import Session

token = open('token_rus.txt').read()

session = Session(token=token, aw_url='https://aw-demo.ru')

In [3]:
df = session.load_model(model_id=13520)

df

,interval,min_date,max_date,summa,donor_id,recurring_yes
0,0,1089,1089,750,7c81052382eb8800f9dc2d98c0b4a724,0
1,0,1088,1088,500,ee5cf36a0e29b21bf9fb6ea33294d30a,0
2,0,1088,1088,1000,6579c30b242298d9e8d245ce29d86727,0
3,0,1088,1088,300,575a6e0939c1410c56ab518e73676488,0
4,0,1087,1087,500,21c98cb768103e6f65eef7fc343907de,0
...,...,...,...,...,...,...
52984,0,3,3,300,0e6adab328356c5a17eb562398030a25,0
52985,0,3,3,150,750d500c02b2d16839f2fd443486040e,0
52986,0,65,65,300,29ff8fb2d364a46eb05e52a134371130,0
52987,0,18,18,100,c34a2547e40347a44c873b48e59fd35d,0


In [4]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52989 entries, 0 to 52988
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interval       52989 non-null  int64 
 1   min_date       52989 non-null  int64 
 2   max_date       52989 non-null  int64 
 3   summa          52989 non-null  int64 
 4   donor_id       52989 non-null  object
 5   recurring_yes  52989 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 2.4+ MB


interval         0
min_date         0
max_date         0
summa            0
donor_id         0
recurring_yes    0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool


CAT_FEATURES = ['donor_id']

FEATURES = [
    'interval',
    'min_date',
    'max_date',
    'summa',
] + CAT_FEATURES

TARGET = 'recurring_yes'
IGNORED_FEATURES = ['donor_id']


def fit_model(df, random_state=1):
    """ 
    Процедура обучения ML-модели 
    """

    X_train, X_val, y_train, y_val = train_test_split(df[FEATURES], df[TARGET], test_size=0.2, random_state=random_state )
    
    train_pool = Pool(X_train, y_train, cat_features=CAT_FEATURES)
    eval_pool = Pool(X_val, y_val, cat_features=CAT_FEATURES)

    model = CatBoostClassifier(random_seed=random_state, learning_rate=0.005, iterations = 2000, ignored_features = df[IGNORED_FEATURES])
    model.fit(train_pool, eval_set=eval_pool, plot=True, verbose=False, use_best_model=True)

    return model

In [12]:
# Последняя ячейка
experiment_id = session.mlflow.setup_experiment('Modul4__rustambash7_demo')

with session.mlflow.start_run(experiment_id=experiment_id):
    model = fit_model(df)
    session.mlflow.log_metrics(model.get_best_score()['validation'])
    session.mlflow.catboost.log_model(model, 'model', registered_model_name='Modul4__rustambash7_demo')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Registered model 'Modul4__rustambash7_demo' already exists. Creating a new version of this model...
2024/12/11 07:46:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Modul4__rustambash7_demo, version 2
Created version '2' of model 'Modul4__rustambash7_demo'.
